In [34]:
import numpy as np
import keras
np.random.seed(123)  
import tensorflow as tf
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import glob
from matplotlib import pyplot as plt
from keras.callbacks import TensorBoard


%matplotlib inline
from keras.datasets import mnist

batch_size = 20
num_classes = 3
epochs = 5


def batch_process(image_generator,dir):
        return  image_generator.flow_from_directory(
        dir,  # this is the target directory
        target_size=(28, 28),  # all images will be resized to 28X28
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

def train_generator():   
    image_generator= ImageDataGenerator(
                                rescale=1./255,
                                shear_range=0.2,
                                zoom_range=0.2,
                                rotation_range=10,
                                data_format="channels_last",         
                                horizontal_flip=True)
    return batch_process(image_generator,'images/training')  

def validation_generator():
    test_datagen = ImageDataGenerator(rescale=1./255,
                                shear_range=0.2,
                                zoom_range=0.2,
                                rotation_range=10,
                                data_format="channels_last",         
                                horizontal_flip=True)
    return batch_process(test_datagen,'images/testing')

  

def get_model(retrain):           
   
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28, 28,1),
                 data_format='channels_last'))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.45))
    model.add(Dense(num_classes, activation='softmax'))    
    return model

from time import time

def process_model(retrain):
    model=get_model(retrain);
    model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
    
    tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

    model.fit_generator(
        train_generator(),
        steps_per_epoch=1000 // batch_size,
        epochs=epochs,
        validation_data=validation_generator(),
        callbacks=[tensorboard]
        )
    model.save('model.h5')
    

In [35]:
def main():
     print("Training the model..")
     process_model(False)
     
    
if __name__ == "__main__":
    main()    

Training the model..
Instructions for updating:
Use the retry module or similar alternatives.
Found 36 images belonging to 3 classes.
Found 14 images belonging to 3 classes.
Epoch 1/5
50/50 [==============================] - 17s 346ms/step - loss: 0.9079 - acc: 0.5745 - val_loss: 0.9152 - val_acc: 0.5714
Epoch 2/5
50/50 [==============================] - 15s 308ms/step - loss: 0.3328 - acc: 0.8735 - val_loss: 1.1624 - val_acc: 0.5714
Epoch 3/5
50/50 [==============================] - 15s 292ms/step - loss: 0.1491 - acc: 0.9663 - val_loss: 1.6437 - val_acc: 0.4286
Epoch 4/5
50/50 [==============================] - 14s 290ms/step - loss: 0.0739 - acc: 0.9845 - val_loss: 1.5820 - val_acc: 0.5714
Epoch 5/5
50/50 [==============================] - 15s 307ms/step - loss: 0.0312 - acc: 0.9927 - val_loss: 1.7528 - val_acc: 0.6429
